<a href="https://colab.research.google.com/github/Madaa73/SMS_spam-ham/blob/main/Copia_de_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Cargar el conjunto de datos de entrenamiento y prueba
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=["label", "message"])


In [ ]:
# Cargar los datos usando pandas
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label','message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label','message'])

# Veamos algunas filas para confirmar la carga
print("Datos de entrenamiento:")
print(train_df.head())
print("\nDatos de validación:")
print(test_df.head())

# Mapear las etiquetas a números: 0 para 'ham', 1 para 'spam'
label_mapping = {'ham': 0, 'spam': 1}
train_df['label_num'] = train_df['label'].map(label_mapping)
test_df['label_num'] = test_df['label'].map(label_mapping)

# Extraer los mensajes y las etiquetas
X_train = train_df['message'].values
y_train = train_df['label_num'].values

X_test = test_df['message'].values
y_test = test_df['label_num'].values

# Preprocesamiento del texto: tokenización y padding.
# Definir parámetros para el tokenizador
max_words = 10000   # máximo número de palabras a tener en cuenta
oov_token = "<OOV>" # token para palabras fuera del vocabulario

tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_words, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)  # Entrena el tokenizador en los mensajes de entrenamiento

# Convertir textos a secuencias de enteros
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

# Definir una longitud máxima para los mensajes. Por ejemplo, podemos usar 50 tokens.
max_length = 50

# Aplicar padding a las secuencias para que tengan la misma longitud
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

# Mostrar forma de los datos preprocesados
print("\nForma de train_padded:", train_padded.shape)
print("Forma de test_padded:", test_padded.shape)


In [ ]:
# Definir dimensiones para la capa de embedding
embedding_dim = 16

# Construcción del modelo MLP
model = keras.Sequential([
    # Capa de embedding para transformar índices en vectores densos
    keras.layers.Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_length),
    # Agrupación global para aplanar la secuencia (opción alternativa es Flatten, pero GlobalAveragePooling1D reduce la dimensionalidad)
    keras.layers.GlobalAveragePooling1D(),
    # Capa oculta densa con activación ReLU
    keras.layers.Dense(24, activation='relu'),
    # Capa de salida con una neurona y activación sigmoid para predecir probabilidad
    keras.layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo con función de pérdida y optimizador adecuados para clasificación binaria
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, max_length))
model.summary()



In [ ]:
# Definir el número de épocas para entrenar
num_epochs = 15

# Entrenar el modelo utilizando el conjunto de entrenamiento y validándolo con el conjunto de validación
history = model.fit(train_padded, y_train,
                    epochs=num_epochs,
                    validation_data=(test_padded, y_test),
                    verbose=2)


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # Convertir el mensaje a secuencia usando el tokenizador entrenado
    sequence = tokenizer.texts_to_sequences([pred_text])
    # Aplicar padding a la secuencia para que tenga la misma longitud
    padded = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    # Realizar la predicción: la salida es un array, extraemos el primer valor
    prob = model.predict(padded)[0][0]
    # Definir la etiqueta: si la probabilidad es menor a 0.5, es "ham"; de lo contrario, "spam"
    label = "ham" if prob < 0.5 else "spam"
    # Retornar la probabilidad y la etiqueta en una lista
    return [prob, label]


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)

    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
